# Imports and path setup




In [99]:
!git clone https://github.com/SIDN-IAP/global-model-repr.git tutorial_code
!pip install transformers==2.1
!pip install spacy ftfy==4.4.3
!python -m spacy download en

import torch
from transformers import BertTokenizer, BertModel
import numpy as np
import sys
# sys.path.append('global-model-repr/')
sys.path.append('/content/tutorial_code')
# sys.path.append('..')
from probing.utils import get_sentence_repr, get_model_and_tokenizer, get_pos_data

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    print("Change runtime type to include a GPU.")  
    device = torch.device('cpu')
print("device:", device)

fatal: destination path 'tutorial_code' already exists and is not an empty directory.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
device: cuda


# Get data for part-of-speech tagging
A probing experiment requires supervised data with linguistic annotation for the property we wish to study. We will use part-of-speech (POS) tagging, a classical problem in NLP. We will use (a portion of) the English Web dependency treebank from the Universal Dependencies project (https://universaldependencies.org/). The dataset comes with POS information, morphological features (tense, gender, number, etc.), and dependency labels (subject, object, etc.), so it can be used to stufy various aspects of language. 

In [100]:
train_sentences, train_labels, test_sentences, test_labels, _, _, label2index = get_pos_data("/content/tutorial_code/probing", frac=0.1)
# train_sentences, train_labels, test_sentences, test_labels, _, _, label2index = get_pos_data("../probing", frac=0.1)
num_labels = len(label2index)
print("Training sentences:", len(train_sentences), "Test sentences:", len(test_sentences))
print("Unique labels:", num_labels)

Training sentences: 1254 Test sentences: 208
Unique labels: 17


# Set up model
A probing experiment also requires a probing model, also known as an auxiliary classifier. Here we define a simple linear classifier, which takes a word representation as input and applies a linear transformation to map it to the label space. 

We also need a pre-model deep neural network to study. We will use the popular BERT model (https://www.aclweb.org/anthology/N19-1423.pdf), available via the HuggingFace Transformers library (https://huggingface.co/transformers/).  The library provides a number of other models that you can easily experiment with thanks to the unified API. 

In [0]:
class Classifier(torch.nn.Module):
    
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, input):
        output = self.linear(input)
        return output
    

class NonlinearClassifier(torch.nn.Module):
    
    def __init__(self, input_dim, output_dim):
        super(NonlinearClassifier, self).__init__()
        
        self.input2hidden = torch.nn.Linear(input_dim, input_dim)
        self.hidden2output = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, input):
        hidden = self.input2hidden(input)
        output = self.hidden2output(hidden)
        return output
    
    
def build_classifier(emb_dim, num_labels, device='cpu'):

    classifier = Classifier(emb_dim, num_labels).to(device)
    criterion = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(classifier.parameters())

    return classifier, criterion, optimizer


def build_nonlinear_classifier(emb_dim, num_labels, device='cpu'):

    classifier = NonlinearClassifier(emb_dim, num_labels).to(device)
    criterion = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(classifier.parameters())

    return classifier, criterion, optimizer


model_name = 'bert-base-cased'
# get model and tokenizer from Transformers
model, tokenizer, sep, emb_dim = get_model_and_tokenizer(model_name, device)
# build classifier
classifier, criterion, optimizer = build_classifier(emb_dim, num_labels, device)

In [102]:
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [103]:
print(classifier)

Classifier(
  (linear): Linear(in_features=768, out_features=17, bias=True)
)


# Train
Given a pre-trained model, a probing classifier, and supervised linguistic annotations, we can run a probing experiment. First, we'll define a training function that trains the classifier on the linguistic annotations. This is a simple implementation, but one could implement various checks like early stopping on a development set, etc. 

In [0]:
def train(num_epochs, train_representations, train_labels, 
          model, tokenizer, sep, model_name, device, 
          classifier, criterion, optimizer, batch_size=32):
    
    num_total = train_representations.shape[0] 
    for i in range(num_epochs):
        total_loss = 0.
        num_correct = 0.
        for batch in range(0, num_total, batch_size):
            batch_repr = train_representations[batch: batch+batch_size]
            batch_labels = train_labels[batch: batch+batch_size]

            optimizer.zero_grad()
            
            out = classifier(batch_repr)
            pred = out.max(1)[1]
            num_correct += pred.long().eq(batch_labels.long()).cpu().sum().item()
            loss = criterion(out, batch_labels)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()
#         print('Training epoch: {}, loss: {}, accuracy: {}'.format(i, total_loss/num_total, num_correct/num_total))
    return total_loss/num_total, num_correct/num_total

# Evaluate
Given the trained classifier, we'll evaluate its performance on the test set. 

In [0]:
def evaluate(test_representations, test_labels, 
             model, tokenizer, sep, model_name, device, 
             classifier, criterion, batch_size=32):
    
    num_correct = 0.
    num_total = test_representations.shape[0]
    total_loss = 0.
    with torch.no_grad():
        for batch in range(0, num_total, batch_size):
            batch_repr = test_representations[batch: batch+batch_size]
            batch_labels = test_labels[batch: batch+batch_size]
            
            out = classifier(batch_repr)
            pred = out.max(1)[1]
            num_correct += pred.long().eq(batch_labels.long()).cpu().sum().item()
            total_loss += criterion(out, batch_labels)

#     print('Testing loss: {}, accuracy: {}'.format(total_loss/num_total, num_correct/num_total))
    return total_loss/num_total, num_correct/num_total

# Generate representations with pretrained model
Here we collect representations from the pre-trained model. We also apply a few data transformations for convenience. 
The end result is `train_sentence_representations`, a list of tensors, where each tensor has representations from one layer in the deep model. Each tensor has dimensions num_word in the corpus x representation dimensionality. 

In [0]:
# top-level list: sentences, second-level lists: layers, third-level tensors of num_words x representation_dim
train_sentence_representations = [get_sentence_repr(sentence, model, tokenizer, sep, model_name, device) 
                                  for sentence in train_sentences]
test_sentence_representations = [get_sentence_repr(sentence, model, tokenizer, sep, model_name, device) 
                                  for sentence in test_sentences]

# top-level list: layers, second-level lists: sentences
train_sentence_representations = [list(l) for l in zip(*train_sentence_representations)]
test_sentence_representations = [list(l) for l in zip(*test_sentence_representations)]                           

# concatenate all word represenations
train_representations_all = [torch.tensor(np.concatenate(train_layer_representations, 0)).to(device) for train_layer_representations in train_sentence_representations]
test_representations_all = [torch.tensor(np.concatenate(test_layer_representations, 0)).to(device) for test_layer_representations in test_sentence_representations]
# concatenate all labels
train_labels_all = torch.tensor(np.concatenate(train_labels, 0)).to(device)
test_labels_all = torch.tensor(np.concatenate(test_labels, 0)).to(device)

# Experiment 1: Evaluate representation for POS quality
In this experiment, we train and evaluate a classifier on the top-level representations of BERT on the task of POS tagging. The test accuracy can be thought of as a measure of the quality of the representations for the POS property. 

In [107]:
# Take final layer representations
train_representations = train_representations_all[-1]
test_representations = test_representations_all[-1]

# train
train_loss, train_accuracy = train(10, train_representations, train_labels_all, 
          model, tokenizer, sep, model_name, device, 
          classifier, criterion, optimizer)
# test
test_loss, test_accuracy = evaluate(test_representations, test_labels_all, 
         model, tokenizer, sep, model_name, device, 
         classifier, criterion)
print("Train accuracy: {}, Test accuracy: {}".format(train_accuracy, test_accuracy))

Train accuracy: 0.9267587893157485, Test accuracy: 0.9177355152587351


# Experiment 2: Compare representation quality across layers
One of the major questions in neural network interpretability is how information is organized in different parts of the deep model, such as its layers. Here we train and evaluate a separate classifier per each layer. Notice the test accuracy results for this task, and how deeper is not always better in our case. 

In [108]:
num_layers = len(train_representations_all)
train_accs, test_accs = [], []
for l in range(num_layers):
    # build new classifier for every layer experiment
    classifier, criterion, optimizer = build_classifier(emb_dim, num_labels, device)
    # get layer representation 
    train_representations = train_representations_all[l]
    test_representations = test_representations_all[l]
    
    # train
    train_loss, train_accuracy = train(2, train_representations, train_labels_all, 
          model, tokenizer, sep, model_name, device, 
          classifier, criterion, optimizer)
    train_accs.append(train_accuracy)
    # test
    test_loss, test_accuracy = evaluate(test_representations, test_labels_all, 
         model, tokenizer, sep, model_name, device, 
         classifier, criterion)
    test_accs.append(test_accuracy)
    print("layer: {}, train accuracy: {}, test accuracy: {}".format(l, train_accuracy, test_accuracy))

layer: 0, train accuracy: 0.8645945245063535, test accuracy: 0.8474126492702344
layer: 1, train accuracy: 0.8843403845441411, test accuracy: 0.8779301194161875
layer: 2, train accuracy: 0.9253139702885933, test accuracy: 0.9298982750995135
layer: 3, train accuracy: 0.9303523135627755, test accuracy: 0.9314462627156126
layer: 4, train accuracy: 0.9329085318415885, test accuracy: 0.9287925696594427
layer: 5, train accuracy: 0.9321305523654281, test accuracy: 0.930561698363556
layer: 6, train accuracy: 0.9323157855740377, test accuracy: 0.932109685979655
layer: 7, train accuracy: 0.9318712258733746, test accuracy: 0.9250331711632022
layer: 8, train accuracy: 0.9245730374541548, test accuracy: 0.9203892083149049
layer: 9, train accuracy: 0.9149409106064536, test accuracy: 0.9126492702344096
layer: 10, train accuracy: 0.9036416848812655, test accuracy: 0.8998230871295887
layer: 11, train accuracy: 0.8949727707183344, test accuracy: 0.8960636886333481
layer: 12, train accuracy: 0.86440929129

# Experiment 3: Non-linear classifier
Does the probing accuracy depend on the probing model? We have previously trained a linear probing classifier. Here we train a non-linear classifier with one hidden layer. Does the layer-wise pattern change with a different probing model? What does this tell us about the information encoded in the model's internal representations?

In [109]:
num_layers = len(train_representations_all)
train_accs, test_accs = [], []
for l in range(num_layers):
    # build non-linear classifier
    classifier, criterion, optimizer = build_nonlinear_classifier(emb_dim, num_labels, device)
    # get layer representation 
    train_representations = train_representations_all[l]
    test_representations = test_representations_all[l]
    
    # train
    train_loss, train_accuracy = train(2, train_representations, train_labels_all, 
          model, tokenizer, sep, model_name, device, 
          classifier, criterion, optimizer)
    train_accs.append(train_accuracy)
    # test
    test_loss, test_accuracy = evaluate(test_representations, test_labels_all, 
         model, tokenizer, sep, model_name, device, 
         classifier, criterion)
    test_accs.append(test_accuracy)
    print("layer: {}, train accuracy: {}, test accuracy: {}".format(l, train_accuracy, test_accuracy))

layer: 0, train accuracy: 0.8586300151891231, test accuracy: 0.8255196815568333
layer: 1, train accuracy: 0.8770421961249213, test accuracy: 0.8356921716054843
layer: 2, train accuracy: 0.9192753676879191, test accuracy: 0.9038036267138434
layer: 3, train accuracy: 0.9241284777534916, test accuracy: 0.8993808049535603
layer: 4, train accuracy: 0.9256844367058126, test accuracy: 0.9007076514816453
layer: 5, train accuracy: 0.9276108620753528, test accuracy: 0.8960636886333481
layer: 6, train accuracy: 0.9282406549846256, test accuracy: 0.891640866873065
layer: 7, train accuracy: 0.9255362501389249, test accuracy: 0.9060150375939849
layer: 8, train accuracy: 0.9220168191753417, test accuracy: 0.8889871738168952
layer: 9, train accuracy: 0.9131256251620791, test accuracy: 0.8792569659442725
layer: 10, train accuracy: 0.9027896121216612, test accuracy: 0.8679787704555506
layer: 11, train accuracy: 0.8947134442262809, test accuracy: 0.8586908447589562
layer: 12, train accuracy: 0.8789315748

# Experiment 4: Control labels

In this experiment we test to see how much of the good performance from Experiments 2 and 3 actually come from things the POS model learned, and how much of it just comes from the probe model. To test this, we use a method from Hewitt and Liang (https://arxiv.org/pdf/1909.03368.pdf). We make a <i>control task</i> which is unrelated to the POS task and do the same probing procedure on the control task. We then measure the <i>selectivity</i> of layers; the difference between their probed accuracy on the POS task and on the control task. If a layer has learned substantial things about the POS task in particular, it should be much better at the POS task than the control task; i.e. it should have high selectivity.

Following Hewitt and Liang, we use the following control task for POS tagging. Each word identity will be assigned a random POS tag, with the distribution of POS tags weighted according to their actual appearance. Each word identity will always have the same tag every time it appears. We then train and test the layers on predicting this tag from the embedding. Note that this tag is a deterministic function of the word identity, so high selectivity means the embedding actually has forgotten something about the word identity.

How are the selectivity results different from the previous accuracy results? What does this tell us about the model's internal representations? 


In [0]:
import random

vocabulary = set(
    word
      for sentence in (train_sentences + test_sentences)
      for word in sentence
)
# all_labels = sum((x.tolist() for x in train_labels), [])
all_labels = train_labels_all.tolist()
control_map = {word: random.choice(all_labels) for word in vocabulary}

control_train_labels = [torch.tensor([control_map[word] for word in sentence]) for sentence in train_sentences]
control_test_labels = [torch.tensor([control_map[word] for word in sentence]) for sentence in test_sentences]
control_train_labels = torch.tensor(np.concatenate(control_train_labels, 0)).to(device)
control_test_labels = torch.tensor(np.concatenate(control_test_labels, 0)).to(device)


In [111]:
num_layers = len(train_representations_all)
control_train_accs, control_test_accs = [], [] 
for l in range(num_layers):
    classifier, criterion, optimizer = build_nonlinear_classifier(emb_dim, num_labels, device)
    # get layer representation 
    train_representations = train_representations_all[l]
    test_representations = test_representations_all[l]
    
    # train
    train_loss, train_accuracy = train(2, train_representations, control_train_labels, 
          model, tokenizer, sep, model_name, device, 
          classifier, criterion, optimizer)
    control_train_accs.append(train_accuracy)
    # test
    test_loss, test_accuracy = evaluate(test_representations, control_test_labels, 
         model, tokenizer, sep, model_name, device, 
         classifier, criterion)
    control_test_accs.append(test_accuracy)    
    print("layer: {}, train accuracy: {}, test accuracy: {}".format(l, train_accuracy, test_accuracy))

layer: 0, train accuracy: 0.7368577038491461, test accuracy: 0.7293233082706767
layer: 1, train accuracy: 0.7191123624643426, test accuracy: 0.697921273772667
layer: 2, train accuracy: 0.7072203904716038, test accuracy: 0.6844316674038037
layer: 3, train accuracy: 0.691660800948394, test accuracy: 0.6738168951791242
layer: 4, train accuracy: 0.6748045789649169, test accuracy: 0.6565678903140203
layer: 5, train accuracy: 0.6551328122105731, test accuracy: 0.628482972136223
layer: 6, train accuracy: 0.6292372096469455, test accuracy: 0.6180893409995577
layer: 7, train accuracy: 0.6046382395435854, test accuracy: 0.5787262273330385
layer: 8, train accuracy: 0.5839662134627496, test accuracy: 0.5603715170278638
layer: 9, train accuracy: 0.5589597303004482, test accuracy: 0.5367094206103494
layer: 10, train accuracy: 0.5372874448931204, test accuracy: 0.5176912870411322
layer: 11, train accuracy: 0.52209832178713, test accuracy: 0.5119416187527642
layer: 12, train accuracy: 0.50442707368577

In [112]:
for l in range(num_layers):
    print("layer: {}, test selectivity: {}".format(l, test_accs[l] - control_test_accs[l]))

layer: 0, test selectivity: 0.0961963732861566
layer: 1, test selectivity: 0.13777089783281726
layer: 2, test selectivity: 0.21937195931003972
layer: 3, test selectivity: 0.22556390977443608
layer: 4, test selectivity: 0.24413976116762504
layer: 5, test selectivity: 0.2675807164971251
layer: 6, test selectivity: 0.2735515258735073
layer: 7, test selectivity: 0.32728881026094647
layer: 8, test selectivity: 0.3286156567890314
layer: 9, test selectivity: 0.34254754533392306
layer: 10, test selectivity: 0.3502874834144184
layer: 11, test selectivity: 0.34674922600619196
layer: 12, test selectivity: 0.35382574082264484
